### basic chat stream

In [2]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import StructuredTool, ToolException
from typing import List, Annotated
from langchain.agents import load_tools
from langchain_experimental.utilities import PythonREPL
from langgraph.prebuilt import ToolNode


# grobid xml processing

In [3]:
import fitz
import os

In [7]:
d = fitz.open(os.path.join('/home/aditya-ladawa/Aditya/z_projects/aireas/api/static/files/', 'a.pdf'))

In [14]:
os.getcwd()

'/home/aditya-ladawa/Aditya/z_projects/aireas/api'

In [67]:
import os
import xml.etree.ElementTree as ET

# Specify the path to your XML file
file_path = os.path.join('metas', 'a.pdf.tei.xml')

# Parse the XML file
tree = ET.parse(file_path)
root = tree.getroot()

# Now you can iterate through elements or access specific tags
for element in root.iter():
    if element.text == None or element.text!= 'None':
        print(element.text)




	

		

			

				
Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need

				
None

					
None
2 Aug 2023

				

					

						

							
None
Ashish
Vaswani
avaswani@google.com

								
Google Brain Google Research Google Research Google Research University of Toronto Google Brain 31st Conference on Neural Information Processing Systems (NIPS 2017) , Long Beach , CA , USA.
Google Brain
Google Research
Google Research
Google Research
Google Brain
31st Conference on Neural Information Processing Systems (NIPS 2017)
University of Toronto

									
Long Beach
CA
USA

							
None
Noam
Shazeer

								
Google Brain Google Research Google Research Google Research University of Toronto Google Brain 31st Conference on Neural Information Processing Systems (NIPS 2017) , Long Beach , CA , USA.
Google Brain
Google Research
Google Research
Google Resea

In [59]:
import xml.etree.ElementTree as ET

# Load the XML file
file_path = os.path.join('metas', 'a.pdf.tei.xml')
tree = ET.parse(file_path)
root = tree.getroot()

# Define the TEI namespace
namespace = {'tei': 'http://www.tei-c.org/ns/1.0'}

# Function to extract text safely
def get_text_or_none(element, tag):
    elem = element.find(tag, namespace)
    return elem.text if elem is not None else None

# Extract title
title = get_text_or_none(root, './/tei:titleStmt/tei:title')
print("Title:", title)

# Extract publication date
date = get_text_or_none(root, './/tei:publicationStmt/tei:date')
print("Publication Date:", date)

# Extract authors
authors = []
for author in root.findall('.//tei:biblStruct/tei:analytic/tei:author', namespace):
    forename = get_text_or_none(author, 'tei:persName/tei:forename')
    surname = get_text_or_none(author, 'tei:persName/tei:surname')
    authors.append(f"{forename} {surname}")
print("Authors:", authors)

# Extract abstract (if present)
abstract = get_text_or_none(root, './/tei:teiHeader/tei:profileDesc/tei:abstract')
print("Abstract:", abstract)


Title: Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need
Publication Date: 2 Aug 2023
Authors: ['Ashish Vaswani', 'Noam Shazeer', 'Google Brain', 'Niki Parmar', 'Jakob Uszkoreit', 'Llion Jones', 'Aidan Gomez', 'Łukasz Kaiser', 'Chris Dyer', 'Adhiguna Kuncoro', 'Miguel Ballesteros', 'Noah Smith', 'Kaiming He', 'Xiangyu Zhang', 'Shaoqing Ren', 'Jian Sun', 'Sepp Hochreiter', 'Jürgen Schmidhuber', 'Zhongqiang Huang', 'Mary Harper', 'Łukasz Kaiser', 'Samy Bengio', 'Łukasz Kaiser', 'Ilya Sutskever', 'Yoon Kim', 'Carl Denton', 'Luong Hoang', 'Alexander Rush', 'Diederik Kingma', 'Jimmy Ba', 'Mary Mitchell P Marcus', 'Ann Marcinkiewicz', 'Beatrice Santorini', 'David Mcclosky', 'Eugene Charniak', 'Mark Johnson', 'Ankur Parikh', 'Oscar Täckström', 'Dipanjan Das', 'Jakob Uszkoreit', 'Slav Petrov', 'Leon Barrett', 'Romain Thibaux', 'Dan Klein', 'Nitis

In [61]:
from lxml import etree

def extract_sections_and_text(element, namespaces):
    sections = {}
    for child in element:
        # Check for section-like elements (like div, p, section, etc.)
        if child.tag in ['{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}p', '{http://www.tei-c.org/ns/1.0}section']:
            section_name = child.find('{http://www.tei-c.org/ns/1.0}head', namespaces)  # Assuming section names are in <head> tags
            section_name = section_name.text if section_name is not None else "Unnamed Section"
            sections[section_name] = []

            # Collect text from the section
            for para in child.xpath('.//tei:p', namespaces):
                sections[section_name].append(para.text)

    return sections

# Parse the XML file
tree = etree.parse(file_path)
root = tree.getroot()

# Define namespaces if needed
namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}

# Extract sections and text
all_sections = extract_sections_and_text(root, namespaces)

# Output the extracted information
for section, texts in all_sections.items():
    print(f"Section: {section}")
    for text in texts:
        print(text)


# Adaptive RAG

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_groq import ChatGroq
from pprint import pprint
llm = ChatGroq(model='llama-3.1-70b-versatile')


In [3]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
        
    )

structured_llm_router = llm.with_structured_output(RouteQuery)
routing_system_prompt = """You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains vectors of pdf texts of several research papers. Currently these pdfs are in the vector database: ['Attention is all you need', 'MaGVIT: Mased Generative Video transformer', 'SAN: INDUCING METRIZABILITY OF GAN WITH
DISCRIMINATIVE NORMALIZED LINEAR LAYER'].
Use the vectorstore for questions on these topics. Otherwise, use web-search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", routing_system_prompt),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
# print(
#     question_router.invoke(
#         {"question": "Who will the Bears draft first in the NFL draft?"}
#     )
# )
print(question_router.invoke({"question": 'compare performance of SAN with other generative models?'}))

datasource='vectorstore'


In [4]:
# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )
structured_llm_doc_grader = llm.with_structured_output(GradeDocuments)

from team_tools import Qretriever

retriever = Qretriever

# Prompt
doc_grader_system_prompt = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", doc_grader_system_prompt),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

question = 'compare performance of SAN with other generative models?'
retrieval_grader = grade_prompt | structured_llm_doc_grader
docs = retriever.invoke(question)
doc_txt = docs

print(retrieval_grader.invoke({"question": question, "document": doc_txt}))


/home/aditya-ladawa/Aditya/z_projects/aireas/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Started Qdrant client.
Collection 'aireas-local' already exists.
binary_score='yes'


In [5]:
docs

[Document(metadata={'pdf_id': 'SAN.pdf', 'score': 0.7212212}, page_content="GAN -Ex∼µ θ [log ς(f (x))] 1 -ς(f (x)) Non-saturating GAN Ex∼µ θ [log ς(1 -f (x))] ς(f (x)) p: A key strength of SAN is that the model can be trained with just two small modifications to existing GAN implementations (see Fig. p: ref: 4 formula: (x) = w ⊤ φ L h φ <L (x) with w φ L ∈ S D-1 p: , where φ l is the parameter for the lth layer. The second is to use the maximization problem defined in Eq. ( p: ref: 15 p: Comparison with other generative models. First, we compare SAN with GAN. In GAN, other than Wasserstein GAN, the optimal discriminator is known to involve a dilemma: it induces a certain dissimilarity but leads to exploding and vanishing gradients p: ref: (Arjovsky et al., 2017; ref: Lin et al., 2021) ref: (Deshpande et al., 2019; ref: Kolouri et al., 2019 ref: 1 ref: (Nguyen et al., 2021; ref: Chen et al., 2022) div: head: EXPERIMENTS p: We perform experiments with synthetic and image datasets to (1) 

In [6]:

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")


In [7]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

        
# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
pprint(generation)

('SAN outperforms other generative models such as GAN, BigGAN, and StyleGAN-XL '
 'in terms of FID (Fréchet Inception Distance) and IS (Inception Score) '
 'metrics. Specifically, SAN achieves lower FID scores and higher IS scores '
 'compared to its counterparts. For example, BigSAN achieves a FID score of '
 '14.45±0.58, while BigGAN achieves a score of 17.16±1.34.')


In [8]:
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough


rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke(question)

{'context': [Document(metadata={'pdf_id': 'SAN.pdf', 'score': 0.7212212}, page_content="GAN -Ex∼µ θ [log ς(f (x))] 1 -ς(f (x)) Non-saturating GAN Ex∼µ θ [log ς(1 -f (x))] ς(f (x)) p: A key strength of SAN is that the model can be trained with just two small modifications to existing GAN implementations (see Fig. p: ref: 4 formula: (x) = w ⊤ φ L h φ <L (x) with w φ L ∈ S D-1 p: , where φ l is the parameter for the lth layer. The second is to use the maximization problem defined in Eq. ( p: ref: 15 p: Comparison with other generative models. First, we compare SAN with GAN. In GAN, other than Wasserstein GAN, the optimal discriminator is known to involve a dilemma: it induces a certain dissimilarity but leads to exploding and vanishing gradients p: ref: (Arjovsky et al., 2017; ref: Lin et al., 2021) ref: (Deshpande et al., 2019; ref: Kolouri et al., 2019 ref: 1 ref: (Nguyen et al., 2021; ref: Chen et al., 2022) div: head: EXPERIMENTS p: We perform experiments with synthetic and image data

In [9]:
# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )
structured_llm_hallucinations_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
hallucination_grader_system_prompt = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", hallucination_grader_system_prompt),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_hallucinations_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='yes')

In [10]:
# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )
structured_llm_answer_grader = llm.with_structured_output(GradeAnswer)

# Prompt
answer_grader_system_prompt = """You are a grader assessing whether an answer addresses / resolves a question \n 
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", answer_grader_system_prompt),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_answer_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='yes')

In [38]:

# Prompt
rewriter_system_prompt = """Rewrite the question to optimize it for retrieval based on its semantic intent."""

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rewriter_system_prompt),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'Here\'s an improved version of the question:\n\n"What are the different methods for transferring video content, including analog and digital formats, from one device or medium to another?"\n\nThis revised question provides more context and specificity, which can help improve retrieval based on its semantic intent. \n\nAlternatively, you could also ask:\n\n* "What video transfer methods are available for converting, copying, or moving video content?"\n* "What are the various techniques used for video migration, duplication, or conversion between different devices or formats?"\n* "How can video content be transferred from one device or medium to another, and what are the different techniques involved?"'

In [10]:
from team_tools import tavily_search_tool

web_search_tool = tavily_search_tool

In [42]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

In [ ]:
pprint(docs)

In [43]:
from langchain.schema import Document


def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": [web_results], "question": question}


### Edges ###


def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    elif source.datasource == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [44]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("web_search", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.add_conditional_edges(
    START,
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
    },
)
workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

In [ ]:
from pprint import pprint

# Run
inputs = {
    "question": "in attention is all you need paper, what is positional encoding?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint(value, indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

# Multi query retriever

In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from pprint import pprint 

load_dotenv()

llm = ChatGroq(model='llama-3.2-90b-vision-preview')

In [8]:
from team_tools import Qretriever
from langchain_core.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List


from langchain_core.messages import AIMessage, AIMessageChunk
def parse(ai_message: AIMessage) -> List[str]:
    """Parse the AI message."""
    # Split the input string by newline, strip whitespace and single quotes, and return a list of queries
    return [line for line in ai_message.content.split('\n')]


# Define the query prompt
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Break down complex queries into simpler ones. Please adhere to the context and topic of the original question. Rephrase the simpler queries to make them efficient for querying a vector database.
    Please try to only generate the most minimum/least number of queries required to answer the question. Do not add repetitive or irrelevant questions.
ONLY PROVIDE THE OUTPUT. List the questions seperated by newlines.

Example input: What is attention mechanism? How can i explain it to a child?
Output:
Explain attention mechanism in deep learning.
Give an example to explain attention mechanism to 5 year old.

Original question: {question}

    """
)


# Initialize the LLM chain
llm_chain = QUERY_PROMPT | llm | parse

question = 'what is the title of paper m.pdf'

pprint(llm_chain.invoke(question))

# Initialize the MultiQueryRetriever
retriever = MultiQueryRetriever(
    retriever=Qretriever, llm_chain=llm_chain
)

# Run the retriever
results = retriever.invoke(question)


# Assuming `documents` is a list of Document objects
refines = [{ 'pdf_id': doc.metadata['pdf_id'], 'score': doc.metadata['score'], 'page_content': doc.page_content } for doc in results]
refines
# print(refines[::-1])
# print('\n')

# llm.invoke(f'{question}\n{refines[::-1]}')

['What is the metadata of paper m.pdf?',
 'Get the title from the retrieved metadata.']


[{'pdf_id': 'SAN.pdf',
  'score': 0.55051386,
  'page_content': '(ICLR), 2023.\nNicolas Bonneel, Julien Rabin, Gabriel Peyr´e, and Hanspeter Pfister. Sliced and radon Wasserstein\nbarycenters of measures. Journal of Mathematical Imaging and Vision, 51(1):22–45, 2015.\nAndrew Brock, Jeff Donahue, and Karen Simonyan. Large scale gan training for high fidelity natural\nimage synthesis. In Proc. International Conference on Learning Representation (ICLR), 2019.\nRobin Chan, Sarina Penquitt, and Hanno Gottschalk. Lu-net: Invertible neural networks based on\nmatrix factorization. arXiv preprint arXiv:2302.10524, 2023.\nTatjana Chavdarova, Gauthier Gidel, Franc¸ois Fleuret, and Simon Lacoste-Julien. Reducing noise\nin gan training with variance reduced extragradient. In Proc. Advances in Neural Information\nProcessing Systems (NeurIPS), volume 32, 2019.\nTatjana Chavdarova, Matteo Pagliardini, Sebastian U Stich, Franc¸ois Fleuret, and Martin Jaggi.\nTaming gans with lookahead-minmax. In Proc. 

In [17]:
# from langchain_core.prompts import PromptTemplate
# from langchain.chains import LLMChain
# from typing import Optional
# from langchain.output_parsers import PydanticOutputParser
# from pydantic import BaseModel, Field

# # Define the output schema for parsing
# class DocumentDetectionOutput(BaseModel):
#     pdf_name: Optional[str] = Field(None, description="The name of the PDF file mentioned in the question, if any.")

# # Define the parsing function to interpret the output
# def parse_document_name(output: str) -> DocumentDetectionOutput:
#     """
#     Parses the LLM output to detect if a specific PDF file name was mentioned.
#     Expected format: 'The pdf_name is: <file_name>'
#     """
#     output_lines = output.strip().splitlines()
#     for line in output_lines:
#         if "pdf_name is:" in line:
#             pdf_name = line.split("pdf_name is:")[-1].strip()
#             return DocumentDetectionOutput(pdf_name=pdf_name if pdf_name else None)
#     return DocumentDetectionOutput(pdf_name=None)

# # Prompt Template to detect document mentions
# DOC_DETECTION_PROMPT = PromptTemplate(
#     input_variables=["question"],
#     template="""Identify if the question pertains to any specific document by name.
#     If a PDF is mentioned, output the name in the format: 'The pdf_name is: <file_name>'.
#     If no specific document is mentioned, respond with: 'No specific document mentioned.'
    
#     Question: {question}
#     """
# )

# # Initialize LLM Chain
# llm_chain = DOC_DETECTION_PROMPT | llm 

# # Example function to use the chain
# def detect_pdf_name(question: str):
#     # Run the LLM chain
#     output = llm_chain.invoke(question)
#     return output


# detect_pdf_name('what is the title of pdf "a"')

# metadata and self query chain

In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(model='llama-3.2-90b-vision-preview')

In [2]:
from langchain_qdrant import QdrantVectorStore
from qdrant_cloud_ops import URL, EMBEDDING_MODEL, QDRANT_API_KEY

# Qdrant Docker is running on localhost, port 6333
url = URL

# Initialize QdrantVectorStore instance
qdrant = QdrantVectorStore.from_existing_collection(
    collection_name="aireas-cloud",
    embedding=EMBEDDING_MODEL,
    api_key=QDRANT_API_KEY,
    url=url,
    prefer_grpc=True,
    content_payload_key="text",  # Main text field
    metadata_payload_key="metadata"  # Metadata field
)



/home/aditya-ladawa/Aditya/z_projects/aireas/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Started Qdrant client.
Collection 'aireas-cloud' already exists.
Index on 'pdf_id' field already exists.
[Document(metadata={'pdf_id': 'a.pdf', '_id': 'e104188a-f98b-4908-b74c-52c1e15d3759', '_collection_name': 'aireas-cloud'}, page_content=''), Document(metadata={'pdf_id': 'a.pdf', '_id': '70ab1791-77cf-42a5-81fe-36e7336b3aab', '_collection_name': 'aireas-cloud'}, page_content=''), Document(metadata={'pdf_id': 'a.pdf', '_id': '1af2ec7c-f855-4e36-92e1-e516bca933f6', '_collection_name': 'aireas-cloud'}, page_content=''), Document(metadata={'pdf_id': 'a.pdf', '_id': '241ceb9f-8a5c-4a58-b082-aa6c24ae4910', '_collection_name': 'aireas-cloud'}, page_content='')]


In [3]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.query_constructors.chroma import ChromaTranslator
from langchain_core.output_parsers import StrOutputParser


metadata_field_info = [
    AttributeInfo(
        name="pdf_id",
        description="The name of the research paper which is in .pdf filetype.",
        type="string",
    ),
]
document_content_description = 'texts of several research papers, academic papers, and scholarly articles.'
retriever = SelfQueryRetriever.from_llm(
    llm, qdrant, document_content_description, metadata_field_info, verbose=True
)
    


# chain = Qretriever | llm | StrOutputParser()



retriever.invoke("what is abstract of san.pdf")


[Document(metadata={'pdf_id': 'san.pdf', '_id': '549f21e5-98f8-4d52-9923-18e2be4a7eaf', '_collection_name': 'aireas-cloud'}, page_content='Axel Sauer, Tero Karras, Samuli Laine, Andreas Geiger, and Timo Aila. Stylegan-t: Unlocking the\npower of gans for fast large-scale text-to-image synthesis. In Proc. International Conference on\nMachine Learning (ICML), 2023.\nSamarth Sinha, Anirudh Goyal, Colin Raffel, and Augustus Odena. Top-k training of GANs: Im-\nproving generators by making critics less critical. In Proc. Advances in Neural Information Pro-\ncessing Systems (NeurIPS), 2020.\nJiaming Song and Stefano Ermon. Bridging the gap between f-gans and wasserstein gans. In Proc.\nInternational Conference on Machine Learning (ICML), pp. 9078–9087, 2020.\nJiaming Song, Chenlin Meng, and Stefano Ermon. Denoising diffusion implicit models. In Proc.\nInternational Conference on Learning Representation (ICLR), 2020.\nYang Song, Chenlin Meng, and Stefano Ermon. Mintnet: Building invertible neur

# qdrant count and search

In [23]:
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Filter, FieldCondition, MatchValue
from team_tools import EMBEDDING_MODEL

# Initialize your Qdrant client
q_client = QdrantClient("http://localhost:6333")  # Adjust the URL as needed

def count_points(qclient: QdrantClient, value_: str, coll_name: str = 'aireas-local', key_: str = 'pdf_id') -> int:
    """
    Count the number of points associated with a specific pdf_id.

    Args:
        qclient (QdrantClient): The Qdrant client instance.
        pdf_id (str): The PDF identifier to count points for.
        coll_name (str): The name of the collection. Default is 'aireas-local'.
        key (str): The key to match against. Default is 'pdf_id'.

    Returns:
        int: The count of points associated with the specified pdf_id.
    """
    count_response = qclient.count(
        collection_name=coll_name,
        count_filter=Filter(
            must=[
                FieldCondition(key=key_, match=MatchValue(value=value_)),
            ]
        ),
        exact=True,
    )
    return int(count_response.count)

def query_points(query_text_: str, key_ : str, qclient: QdrantClient, value_: str, limit: int = 3, coll_name: str = 'aireas-local') -> list:
    """
    Query points from the Qdrant collection based on a pdf_id and a query string.

    Args:
        qclient (QdrantClient): The Qdrant client instance.
        pdf_id (str): The PDF identifier to filter points.
        limit (int): The maximum number of points to return. Default is 3.
        query_text (str): The text to embed for querying. Default is 'references'.
        coll_name (str): The name of the collection. Default is 'aireas-local'.

    Returns:
        list: The results from the query, containing points matching the criteria.
    """
    # Generate the query embedding
    query_embedding = EMBEDDING_MODEL.embed_query(query_text_)
    prefetch_limit = count_points(qclient, value_, coll_name)
    
    # Execute the query
    results = qclient.query_points(
        collection_name=coll_name,
        query=query_embedding,
        prefetch=[
            models.Prefetch(
                filter=models.Filter(
                    must=models.FieldCondition(
                        key=key_,
                        match=models.MatchValue(value=value_),
                    ),
                ),
                limit=prefetch_limit  # Prefetch limit, adjust as needed
            ),
        ],
        limit=limit
    )
    
    return results


# # Usage example
# pdf_id = 'a.pdf'  # Replace with your actual pdf_id
# points_count = count_points(q_client, pdf_id)  # Get the count of points for the specified pdf_id
# print(f"Number of points associated with '{pdf_id}': {points_count}")

# # Query points for the specified pdf_id
# query_results = query_points(q_client, pdf_id)
# query_results


In [9]:
r.points

[ScoredPoint(id='80dd3aa3-35f6-4853-953d-d9ac7bae6c09', version=4, score=0.502875, payload={'pdf_id': 'a.pdf', 'text': 'corpus of english: The penn treebank. Computational linguistics, 19(2):313–330, 1993.\n[26] David McClosky, Eugene Charniak, and Mark Johnson. Effective self-training for parsing. In\nProceedings of the Human Language Technology Conference of the NAACL, Main Conference,\npages 152–159. ACL, June 2006.\n[27] Ankur Parikh, Oscar Täckström, Dipanjan Das, and Jakob Uszkoreit. A decomposable attention\nmodel. In Empirical Methods in Natural Language Processing, 2016.\n[28] Romain Paulus, Caiming Xiong, and Richard Socher. A deep reinforced model for abstractive\nsummarization. arXiv preprint arXiv:1705.04304, 2017.\n[29] Slav Petrov, Leon Barrett, Romain Thibaux, and Dan Klein. Learning accurate, compact,\nand interpretable tree annotation. In Proceedings of the 21st International Conference on\nComputational Linguistics and 44th Annual Meeting of the ACL, pages 433–440. A

In [69]:
# import fitz  # PyMuPDF
# import os
# # Open the PDF file
# pdf_document = fitz.open(os.path.join("static/files/", 'm.pdf'))

# # Initialize a variable to hold all text
# all_text = ""

# # Iterate through each page and extract text
# for page_num in range(len(pdf_document)):
#     page = pdf_document.load_page(page_num)  # Load the page
#     all_text += page.get_text()  # Extract text from the page

# # Close the PDF document
# pdf_document.close()

# # Print all extracted text
# print(all_text)


In [70]:
os.getcwd()

'/home/aditya-ladawa/Aditya/z_projects/aireas/api'

# parallel runnable pass through chain to get metadata

In [ ]:
# from langchain import hub
# from langchain_core.output_parsers import StrOutputParser
# from team_tools import Qretriever

# retriever = Qretriever

# trimmer = trim_messages(
#     max_tokens=120000,
#     strategy="last",
#     token_counter=llm,
#     include_system=True,
# )


# # Prompt
# prompt = hub.pull("rlm/rag-prompt")
# # Post-processing
# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# from langchain_core.runnables import RunnableParallel
# from langchain_core.runnables import RunnablePassthrough

# rag_chain_from_docs = (
#     RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# rag_chain_with_source = RunnableParallel(
#     {"context": retriever, "question": RunnablePassthrough()}
# ).assign(answer=rag_chain_from_docs)

# rag_chain_with_source.invoke("What is attention")

# response = rag_chain_with_source.invoke("What is attention, and how is it realted to SAN pdf")

# # Access the answer and sources
# answer = response['answer']
# sources = [doc.metadata['pdf_id'] for doc in response['context']]


# Qdrant for fastapi

In [ ]:

# # Qdrant client
# @app.on_event("startup")
# async def startup_event():
#     client = connect_to_qdrant()
#     app.state.qdrant_client = client

# @app.on_event("shutdown")
# async def shutdown_event():
#     client = app.state.qdrant_client
#     if client:
#         print("Shutting down Qdrant client")
#         client.close()

# # Qdrant client
# def get_qdrant_client():
#     return app.state.qdrant_client


# Decomposition chain

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are an assistant that decomposes complex questions into simpler, isolated sub-questions. Your task is to break down the given question into the smallest possible set of essential sub-problems or sub-questions, with a limit of 3 sub-questions. Each sub-question should be answerable independently and necessary to address the original question.
If the question demands it then only you need to break down the question into simpler sub-questions.
Respond only with the sub-questions, separated by newline.

Question: {question}
"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [66]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatGroq(model='llama-3.1-70b-versatile')

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "what is the title of a.pdf and also explain me the abstract of this file"
questions = generate_queries_decomposition.invoke({"question":question})

questions

['What is the title of the file named a.pdf?',
 'What is the content of the file a.pdf?',
 'Is there an abstract or summary within the content of the file a.pdf?']

In [73]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [74]:
questions

["What is the title of the PDF file 'a.pdf'?",
 "What is the content of the PDF file 'a.pdf'?",
 "What is the abstract or summary of the content of the PDF file 'a.pdf'?"]

In [69]:
answers

["The title of the PDF file 'a.pdf' cannot be determined from the context provided.",
 "The PDF file 'a.pdf' contains text about a neural network model, specifically a transformer model, including its architecture, components such as encoder and decoder, attention mechanisms, and positional encoding. The text also includes visualizations of attention mechanisms and examples of the model's behavior.",
 'The content cannot be summarized into an abstract as the provided context appears to be a list of references or citations from various research papers, without any narrative or descriptive text to summarize.']

In [70]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_answer = final_rag_chain.invoke({"context":context,"question":question})

In [ ]:
from pprint import pprint

pprint(final_answer)

# search_agent_test_extra

In [ ]:
# search_agent_prompt = '''
# You are a helpful assistant which given a query will use appropriate tool.
# Tools available to you:
# 1. tavily_search_tool - Useful when you want to search something on internet.
# 2. web_scraper_tool - useful when you need to scrape the contents of website when given website link/s.
# 3. arxiv_search_tool - useful when you need to get citations or information on a certain research paper.

# '''
# search_agent = create_react_agent(llm, tools=[tavily_search_tool, web_scraper_tool, arxiv_search_tool], state_modifier=search_agent_prompt)

# inputs = {"messages": [("user", "what is attention mechanism in transformer? what are the recent updates?")]}

# for s in search_agent.stream(inputs, stream_mode="values"):
#   message = s["messages"][-1]
#   if isinstance(message, tuple):
#       print(message)
#   else:
#       message.pretty_print()

# research team

## Research team with each tool as react agent.

In [ ]:


# class SearchTeamState(TypedDict):
#     messages: Annotated[List[BaseMessage], operator.add]
#     team_members: List[str]
#     next: str

# search_agent = create_react_agent(llm, tools=[tavily_search_tool, web_scraper_tool, arxiv_search_tool])
# search_node = functools.partial(agent_node, agent=search_agent, name="Search")

# web_search_agent = create_react_agent(llm, tools=[web_scraper_tool])
# web_search_node = functools.partial(agent_node, agent=web_search_agent, name="WebScraper")

# arxiv_search_agent = create_react_agent(llm, tools=[arxiv_search_tool])
# arxiv_search_node = functools.partial(agent_node, agent=arxiv_search_agent, name="ArxivSearch")

# search_supervisor_agent = create_team_supervisor(
#     llm,
#     """
#         Supervisor Instructions for Task Management

#         You are assigned as the supervisor overseeing a conversation involving the following worker agents:
#         - **TavilySearch**
#         - **WebScraper**
#         - **ArxivSearch**

#         ### Objective
#         Based on the given user request, decide which worker should execute the next task in the workflow. 
#         Each worker will perform its assigned task and return its results along with a status update.

#         ### Guidelines
#         - **Efficiency**: Minimize search steps to save the user's time.
#         - **Completion**: Once all necessary tasks are complete, respond with FINISH.
# """,
#     ["TavilySearch", "WebScraper", "ArxivSearch"],
# )

# search_graph = StateGraph(SearchTeamState)

# search_graph.add_node('TavilySearch', tavily_search_node)
# search_graph.add_node('WebScraper', web_search_node)
# search_graph.add_node('ArxivSearch', arxiv_search_node)
# search_graph.add_node('SearchSupervisor', search_supervisor_agent)

# search_graph.add_edge('TavilySearch', 'SearchSupervisor')
# search_graph.add_edge('WebScraper', 'SearchSupervisor')
# search_graph.add_edge('ArxivSearch', 'SearchSupervisor')

# search_graph.add_conditional_edges(
#     'SearchSupervisor',
#     lambda x: x['next'],
#     {"TavilySearch": "TavilySearch", "WebScraper": "WebScraper", "ArxivSearch": "ArxivSearch", "FINISH": END}
# )

# search_graph.add_edge(START, 'SearchSupervisor')

# search_graph_compile = search_graph.compile()
# search_chain = enter_chain | search_graph_compile 

# from IPython.display import Image, display

# display(Image(search_graph_compile.get_graph(xray=True).draw_mermaid_png()))

# for s in search_chain.stream(
#     "what is a transformer, and tell me the latest updates regardding the vdieo transformer by open ai", {"recursion_limit": 100}
# ):
#     if "__end__" not in s:
#         print(s)
#         print("---")

## research team with react search agent with all search tools

In [ ]:
# class ResearchTeamState(TypedDict):
#     messages: Annotated[List[BaseMessage], operator.add]
#     team_members: List[str]
#     next: str

# rag_agent = create_react_agent(llm, tools=[qdrant_retriever_tool])
# rag_node = functools.partial(agent_node, agent=rag_agent, name="RAGSearcher")

# search_agent = create_react_agent(llm, tools=[tavily_search_tool, web_scraper_tool, arxiv_search_tool])
# search_node = functools.partial(agent_node, agent=search_agent, name="Searcher")

# code_agent = create_react_agent(llm, tools=[repl_tool])
# code_node = functools.partial(agent_node, agent=search_agent, name="Coder")

# research_supervisor = create_team_supervisor(
#     llm,
#     """
#     Supervisor Instructions for Task Management

#     You are assigned as the supervisor overseeing a conversation involving the following worker agents:
    
#     - **RAGSearcher**: This agent specializes in retrieving and analyzing relevant documents from the Qdrant Vector Store, handling specific document embedding searches. Use this agent to retrieve highly relevant sections of a document based on the user’s query, especially if the answer might lie within previously embedded resources.
    
#     - **Searcher**: This agent has access to web search and scraping tools, including **Tavily Search** for Google-based searches, **WebScraper** for targeted data extraction from specific web pages, and **ArxivSearch** for querying academic papers. Use the Searcher to gather new information from external sources or to clarify details that cannot be answered with existing documents.
    
#     - **Coder**: This agent has a code execution environment (**Python REPL tool**) and should be tasked with tasks that require data analysis, code-based computations, or visualizations. Utilize the Coder whenever a solution requires scripting, data processing, or creating custom visualizations.

#     ### Guidelines for Task Management
#     - **Efficiency**: Minimize search steps to save the user’s time by directing queries to the most relevant agent, avoiding redundant searches.
#     - **Task Delegation**: Ensure tasks are appropriately delegated:
#         - Use **RAGSearcher** when user queries involve pre-stored, embedded knowledge.
#             ( - The following research paper texts are currently available in the vector database in embedded form: "Attention is All You Need," "MAGVIT: Masked Generative Video Transformer," and "SAN: Inducing Metrizability of GAN with Discriminative Normalized Linear Layer.")
#         - Use **Searcher** when real-time data or unembedded references are required.
#         - Use **Coder** for tasks that involve data processing or require executable code.
#     - **Relevance and Specificity**: Tailor instructions for each agent based on the complexity and specificity of the task.
#     - **Clear Communication**: Summarize findings and next steps for each agent’s outputs clearly before returning them to the user.

#     ### Supervisor Guidelines
#     - **Optimize Efficiency**: Select only the necessary tools to minimize the number of steps and save the user’s time.
#     - **Focus on Relevance**: Prioritize agents based on the user’s specific needs to gather the most pertinent information.
#     - **Conclude Tasks**: When all necessary information has been retrieved, conclude the workflow with **FINISH**.

#     """,
#     ["RAGSearcher", "Searcher", "Coder"],
# )
# research_team = StateGraph(ResearchTeamState)

# # Add nodes for each agent in the research team
# research_team.add_node('RAGSearcher', rag_node)
# research_team.add_node('Searcher', search_node)
# research_team.add_node('Coder', code_node)
# research_team.add_node('ResearchSupervisor', research_supervisor)

# # Define edges where each agent reports back to the supervisor
# research_team.add_edge('RAGSearcher', 'ResearchSupervisor')
# research_team.add_edge('Searcher', 'ResearchSupervisor')
# research_team.add_edge('Coder', 'ResearchSupervisor')

# # Add conditional edges based on the 'next' field in ResearchTeamState
# research_team.add_conditional_edges(
#     'ResearchSupervisor',
#     lambda x: x['next'],
#     {"RAGSearcher": "RAGSearcher", "Searcher": "Searcher", "Coder": "Coder", "FINISH": END}
# )

# # Define the starting point of the graph
# research_team.add_edge(START, 'ResearchSupervisor')

# # Compile the graph into a chain
# research_team_compile = research_team.compile()
# research_chain = enter_chain | research_team_compile 

# for s in research_chain.stream(
#     "what is the title of a.pdf?", {"recursion_limit": 100}
# ):
#     if "__end__" not in s:
#         print(s)
#         print("---")

## research team but wrong graph

In [ ]:
# from langchain_core.messages import (
#     BaseMessage,
#     HumanMessage,
#     ToolMessage,
#     AIMessage
# )
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# def create_agent(llm, tools, system_message: str):
#     """Create an agent."""
#     prompt = ChatPromptTemplate.from_messages(
#         [
#             (
#                 "system",
#                 "You are a helpful AI assistant."
#                 " Use the provided tools to progress towards answering the question."
#                 " Respond with 'FINISH' when done."
#                 " You have access to the following tools: {tool_names}.\n{system_message}",
#             ),
#             MessagesPlaceholder(variable_name="messages"),
#         ]
#     )
#     prompt = prompt.partial(system_message=system_message)
#     prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
#     return prompt | llm.bind_tools(tools)

# import operator
# from typing import Annotated, Sequence
# from typing_extensions import TypedDict
# from langgraph.prebuilt import ToolNode, tools_condition

# from typing import Literal


# def route_tools(
#     state: ResearchTeamState,
# ):
#     """
#     Use in the conditional_edge to route to the ToolNode if the last message
#     has tool calls. Otherwise, route to the end.
#     """
#     if isinstance(state, list):
#         ai_message = state[-1]
#     elif messages := state.get("messages", []):
#         ai_message = messages[-1]
#     else:
#         raise ValueError(f"No messages found in input state to tool_edge: {state}")
#     if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
#         return "tools"
#     return END



# class ResearchTeamState(TypedDict):
#     messages: Annotated[Sequence[BaseMessage], operator.add]
#     team_members: List[str]
#     next: str

# rag_agent = create_agent(
#     llm,
#     [qdrant_retriever_tool],
#     system_message="You should use qdrant_retriever_tool to retrieve accurate information from vector database.",
# )
# rag_node = functools.partial(agent_node, agent=rag_agent, name="RagAgent")

# search_agent = create_agent(
#     llm,
#     [tavily_search_tool, arxiv_search_tool, web_scraper_tool],
#     system_message="You should use relevant tool out of tavily_search_tool, arxiv_search_tool, web_scraper_tool to fetch information.",
# )
# search_node = functools.partial(agent_node, agent=search_agent, name="SearchAgent")

# code_agent = create_agent(
#     llm,
#     [repl_tool],
#     system_message="You should use this tool if you need to do the math or code anything, even to plot visualizations with matplotlib or seaborn.",
# )
# code_node = functools.partial(agent_node, agent=code_agent, name="CodeAgent")

# research_supervisor = create_team_supervisor(
#     llm,
#     """
#     ### Supervisor Instructions for Task Management

#     You are the supervisor responsible for coordinating the following specialized agents:

#     - **RagAgent**: Utilizes the **Qdrant Retriever Tool** to fetch highly relevant information from a vector database. This agent is ideal for handling user queries that may be answered from pre-embedded content within the database. Deploy RagAgent when the user’s question is likely addressed by specific sections in stored documents.
    
#     - **SearchAgent**: Equipped with **Tavily Search**, **Arxiv Search**, and **WebScraper** tools, this agent performs external web and academic searches. Use SearchAgent to gather real-time information, explore academic references, or extract data from specific web pages as needed. This agent is suitable for expanding beyond embedded content or when additional clarification is needed.
    
#     - **CodeAgent**: Operates within a **Python REPL** environment, capable of handling code execution for data processing, mathematical calculations, or creating visualizations. Use CodeAgent whenever the task involves computational steps, data transformation, or graphical outputs.

#     ### Task Delegation Guidelines
#     - **Efficiency**: Prioritize relevant agents to streamline workflow and minimize unnecessary steps. 
#     - **Delegate by Task Type**:
#         - Assign **RagAgent** for queries concerning content within the vector database.
#         - Select **SearchAgent** for gathering new information or clarifying details that require web searches or external academic references.
#         - Use **CodeAgent** when calculations, data analysis, or visualizations are necessary.
#     - **Optimize for Relevance**: Tailor instructions based on the nature of the user’s query to ensure agents work with specificity and accuracy.

#     ### Task Workflow and Communication
#     - **Clear Summarization**: Summarize each agent's findings clearly and concisely, preparing the information to relay back to the user.
#     - **End Task Flow**: Conclude the task sequence with **FINISH** once all required information has been obtained and is ready for the user.

#     ### Current Embedded Documents
#     - The following research papers are pre-embedded in the vector database for RagAgent to access:
#         - "Attention is All You Need"
#         - "MAGVIT: Masked Generative Video Transformer"
#         - "SAN: Inducing Metrizability of GAN with Discriminative Normalized Linear Layer"

#     """,
#     ["RagAgent", "SearchAgent", "CodeAgent"],
# )


# # Initialize StateGraph for the Research Team
# research_team = StateGraph(ResearchTeamState)

# # Add nodes for each agent and the supervisor
# research_team.add_node("RagAgent", rag_node)
# research_team.add_node("SearchAgent", search_node)
# research_team.add_node("CodeAgent", code_node)
# research_team.add_node("ResearchSupervisor", research_supervisor)

# research_team.add_node("tools", ToolNode(tools=[qdrant_retriever_tool, tavily_search_tool, arxiv_search_tool, web_scraper_tool, repl_tool]))

# # Define edges where each agent reports back to the supervisor
# research_team.add_edge("RagAgent", "ResearchSupervisor")
# research_team.add_edge("SearchAgent", "ResearchSupervisor")
# research_team.add_edge("CodeAgent", "ResearchSupervisor")

# # Add conditional edges based on the 'next' field in ResearchTeamState
# research_team.add_conditional_edges(
#     "ResearchSupervisor",
#     lambda x: x["next"],
#     {"RagAgent": "RagAgent", "SearchAgent": "SearchAgent", "CodeAgent": "CodeAgent", "FINISH": END}
# )

# research_team.add_conditional_edges(
#     "RagAgent",
#     route_tools,
#     {"tools": "tools", END: END}
# )

# research_team.add_conditional_edges(
#     "SearchAgent",
#     route_tools,
#     {"tools": "tools", END: END}
# )

# research_team.add_conditional_edges(
#     "CodeAgent",
#     route_tools,
#     {"tools": "tools", END: END}
# )

# research_team.add_edge("tools", "RagAgent")
# research_team.add_edge("tools", "SearchAgent")
# research_team.add_edge("tools", "CodeAgent")

# # Define the starting point of the graph
# research_team.add_edge(START, "ResearchSupervisor")

# # Compile the graph into a chain
# research_team_compile = research_team.compile()
# research_chain = enter_chain | research_team_compile


# from IPython.display import Image, display

# display(Image(research_team_compile.get_graph(xray=True).draw_mermaid_png()))


# for s in research_team_compile.stream(
#     {'messages': [HumanMessage(
#                  content="what is attention mechanism"
#             )]}
# ):
#     if "__end__" not in s:
#         print(s)
#         print("---")

# for tool_call in a.tool_calls:
#     selected_tool = {"retrieve_research_paper_texts": qdrant_retriever}[tool_call["name"].lower()]
#     tool_msg = selected_tool.invoke(tool_call)
#     print(tool_msg)



## research team but with rewoo

In [ ]:
prompt = """
For the following task, create a series of plans that can solve the problem step by step. Each plan should break down the task into smaller, simpler sub-questions that efficiently utilize the available tools. Stick to the context of the original question to ensure clarity and coherence in the steps. Each plan should use the minimum number of steps necessary to provide a sufficient answer to the user.

For each plan, specify the external tool to retrieve the necessary evidence and store this evidence in a variable #E that can be used by later tools (e.g., Plan, #E1, Plan, #E2, Plan, ...).

Tools available:
qdrant_retriever_tool[input]: Retrieves documents or relevant research papers from an embedded vector database, useful for finding answers based on prior knowledge or previously indexed content (use when the task relates to research papers).
repl_tool[input]: A Python REPL (Read-Eval-Print Loop) for executing code, data analysis, or visualizations.
arxiv_search_tool[input]: Searches for academic papers and research on Arxiv. Use it to gather specific research papers or academic studies.
tavily_search_tool[input]: Performs Google searches, useful for retrieving quick information from external websites and sources (use for general web-based queries).

Routing Guidance:
For questions related to research papers or prior knowledge, use the qdrant_retriever_tool to retrieve evidence from the vectorstore.
For other questions (especially those requiring external web searches or specific data from web pages), use tavily_search_tool, arxiv_search_tool, or web_scraper_tool as appropriate.

Each plan should be followed by only one #E.

Examples:

Task: Retrieve the latest news on the Australian Open.
   Plan: Search for the latest updates on the 2024 Australian Open via Tavily Search. #E1 = tavily_search_tool[2024 Australian Open news]
   Plan: Retrieve additional information from relevant research papers if available in the vector store using RAG search. #E2 = qdrant_retriever_tool[Australian Open 2024]
   Plan: Use the gathered information to generate a summary of the latest updates. #E3 = LLM[Summarize #E1, #E2]

Task: Find the latest advancements in Generative AI models.
   Plan: Search for recent papers on Arxiv related to Generative AI advancements. #E1 = arxiv_search_tool[Generative AI advancements]
   Plan: Retrieve relevant research papers related to Generative AI from the vector database using RAG search. #E2 = qdrant_retriever_tool[Generative AI]
   Plan: Use the retrieved papers to summarize key insights and developments. #E3 = LLM[Summarize #E1, #E2]

Begin! Provide only necessary output and nothing else. Create a plan such that each sub-question is answered.
Please create the minimum number of plans necessary to answer the task efficiently. Each Plan should be followed by only one #E.

Task: {task}
"""
task = "what is attention mecahnism, and also explain multi collinearity, explain hallucaination to a child in context of deep learning"

from typing import List
from typing_extensions import TypedDict
from pprint import pprint

class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str
    
result = llm.invoke(prompt.format(task=task))   
pprint(result.content)


import re

from langchain_core.prompts import ChatPromptTemplate

# Regex to match expressions of the form E#... = ...[...]
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"
prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
planner = prompt_template | trimmer | llm


def get_plan(state: ReWOO):
    task = state["task"]
    result = planner.invoke({"task": task})
    # Find all matches in the sample text
    matches = re.findall(regex_pattern, result.content)
    return {"steps": matches, "plan_string": result.content}

def _get_current_task(state: ReWOO):
    if "results" not in state or state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1

def tool_execution(state: ReWOO):
    """Worker node that executes the tools of a given plan."""
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state["steps"][_step - 1]
    _results = (state["results"] or {}) if "results" in state else {}
    
    # Replace placeholders in tool_input with results from previous steps
    for k, v in _results.items():
        tool_input = tool_input.replace(k, v)
    
    # Execute the correct tool based on the provided tool name
    if tool == "tavily_search_tool":
        result = tavily_search_tool.invoke(tool_input)
    elif tool == "arxiv_search_tool":
        result = arxiv_search_tool.invoke(tool_input)
    elif tool == "repl_tool":
        result = repl_tool.invoke(tool_input)
    elif tool == "qdrant_retriever_tool":
        result = qdrant_retriever_tool.invoke(tool_input)
    elif tool == "LLM":
        result = llm.invoke(tool_input)  # assuming 'llm' is your pretrained LLM object
    else:
        raise ValueError(f"Unknown tool: {tool}")
    
    # Store the result in the state
    _results[step_name] = str(result)
    return {"results": _results}

solve_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and \
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might \
contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. After going through all answers provide short answers to each question, in w to 3 lines.

Task: {task}
Response:"""


def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = (state["results"] or {}) if "results" in state else {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            step_name = step_name.replace(k, v)
        plan += f"Plan: {_plan}\n{step_name} = {tool}[{tool_input}]"
    prompt = solve_prompt.format(plan=plan, task=state["task"])
    result = llm.invoke(prompt)
    return {"result": result.content}

def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        # We have executed all tasks
        return "solve"
    else:
        # We are still executing tasks, loop back to the "tool" node
        return "tool"
from langgraph.graph import END, StateGraph, START

# Define your plan and solve functions (not shown in your code, assuming they're defined elsewhere)
graph = StateGraph(ReWOO)
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)
graph.add_edge("plan", "tool")
graph.add_edge("solve", END)
graph.add_conditional_edges("tool", _route)
graph.add_edge(START, "plan")

# Compile the graph
app = graph.compile()

# for s in app.stream({"task": task}):
#     print(s)
#     print("---")

# # Print out the final result
# pprint(s["solve"]["result"])

## basic react

## failed network

In [ ]:
# from langchain_core.messages import (
#     BaseMessage,
#     HumanMessage,
#     ToolMessage,
# )
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# from langgraph.graph import END, StateGraph, START


# def create_agent(llm, tools, system_message: str):
#     """Create an agent."""
#     prompt = ChatPromptTemplate.from_messages(
#         [
#             (
#                 "system",
#                 "You are a helpful AI assistant, collaborating with other assistants."
#                 " Use the provided tools to progress towards answering the question."
#                 " If you are unable to fully answer, that's OK, another assistant with different tools "
#                 " will help where you left off. Execute what you can to make progress."
#                 " If you or any of the other assistants have the final answer or deliverable,"
#                 " prefix your response with FINAL ANSWER so the team knows to stop."
#                 " You have access to the following tools: {tool_names}.\n{system_message}",
#             ),
#             MessagesPlaceholder(variable_name="messages"),
#         ]
#     )
#     prompt = prompt.partial(system_message=system_message)
#     prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
#     return prompt | llm.bind_tools(tools)

# import operator
# from typing import Annotated, Sequence
# from typing_extensions import TypedDict

# from langchain_openai import ChatOpenAI


# # This defines the object that is passed between each node
# # in the graph. We will create different nodes for each agent and tool
# class AgentState(TypedDict):
#     messages: Annotated[Sequence[BaseMessage], operator.add]
#     sender: str

# import functools

# from langchain_core.messages import AIMessage


# # Helper function to create a node for a given agent
# def agent_node(state, agent, name):
#     result = agent.invoke(state)
#     # We convert the agent output into a format that is suitable to append to the global state
#     if isinstance(result, ToolMessage):
#         pass
#     else:
#         result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
#     return {
#         "messages": [result],
#         # Since we have a strict workflow, we can
#         # track the sender so we know who to pass to next.
#         "sender": name,
#     }


# # Research agent and node
# research_agent = create_agent(
#     llm,
#     [tavily_search_tool, web_scraper_tool, arxiv_search_tool],
#     system_message="You should provide accurate data for the chart_generator to use.",
# )
# research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# # Qdrant retriever agent and node
# qdrant_retriever_agent = create_agent(
#     llm,
#     [qdrant_retriever_tool],
#     system_message="Retrieve and analyze the most relevant information.",
# )
# qdrant_retriever_node = functools.partial(agent_node, agent=qdrant_retriever_agent, name="qdrant_retriever")

# # Chart generator agent and node
# chart_agent = create_agent(
#     llm,
#     [repl_tool],
#     system_message="Any charts you display will be visible by the user.",
# )
# chart_node = functools.partial(agent_node, agent=chart_agent, name="chart_generator")

# from langgraph.prebuilt import ToolNode

# # Single ToolNode with all tools combined
# tool_node = ToolNode([
#     tavily_search_tool, 
#     web_scraper_tool, 
#     arxiv_search_tool, 
#     qdrant_retriever_tool, 
#     repl_tool
# ])

# # Either agent can decide to end
# from typing import Literal


# def router(state):
#     # This is the router
#     messages = state["messages"]
#     last_message = messages[-1]
#     if last_message.tool_calls:
#         # The previous agent is invoking a tool
#         return "call_tool"
#     if "FINAL ANSWER" in last_message.content:
#         # Any agent decided the work is done
#         return END
#     return "continue"

# workflow = StateGraph(AgentState)

# # Add nodes for each agent
# workflow.add_node("Researcher", research_node)  # Research agent with Tavily, Web Scraper, Arxiv tools
# workflow.add_node("chart_generator", chart_node)  # Chart generator agent with REPL tool
# workflow.add_node("qdrant_retriever", qdrant_retriever_node)  # Qdrant retriever agent

# # Add a single tool node combining all tools
# workflow.add_node("all_tools", tool_node)

# # Define the conditional edges for routing between nodes
# workflow.add_conditional_edges(
#     "Researcher",
#     router,
#     {"continue": "chart_generator", "call_tool": "all_tools", END: END},
# )
# workflow.add_conditional_edges(
#     "chart_generator",
#     router,
#     {"continue": "Researcher", "call_tool": "all_tools", END: END},
# )
# workflow.add_conditional_edges(
#     "qdrant_retriever",
#     router,
#     {"continue": "Researcher", "call_tool": "all_tools", END: END},
# )

# # Routing back to the invoking agent
# workflow.add_conditional_edges(
#     "all_tools",
#     # Route back to the original agent that invoked the tool
#     lambda x: x["sender"],
#     {
#         "Researcher": "Researcher",
#         "chart_generator": "chart_generator",
#         "qdrant_retriever": "qdrant_retriever",
#     },
# )

# # Define the starting point of the workflow
# workflow.add_edge(START, "Researcher")

# # Compile the workflow graph
# graph = workflow.compile()

# # from IPython.display import Image, display

# # try:
# #     display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
# # except Exception:
# #     # This requires some extra dependencies and is optional
# #     pass


# events = graph.stream(
#     {
#         "messages": [
#             HumanMessage(
#                 content='what is attention, how can i explain it to a 60 year old, also tell exlain me the abstratc of a.pdf'
#             )
#         ],
#     },
#     # Maximum number of steps to take in the graph
#     {"recursion_limit": 150},
# )
# for s in events:
#     print(s)
#     print("----")

## research team with wrong reWoo

In [ ]:
# import re

# from langchain_core.prompts import ChatPromptTemplate

# # Regex to match expressions of the form E#... = ...[...]
# regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"
# prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
# planner = prompt_template | llm


# def get_plan(state: ReWOO):
#     task = state["task"]
#     result = planner.invoke({"task": task})
#     # Find all matches in the sample text
#     matches = re.findall(regex_pattern, result.content)
#     return {"steps": matches, "plan_string": result.content}

# def _get_current_task(state: ReWOO):
#     if "results" not in state or state["results"] is None:
#         return 1
#     if len(state["results"]) == len(state["steps"]):
#         return None
#     else:
#         return len(state["results"]) + 1

# def agent_execution(state: ReWOO):
#     """Worker node that executes the tools of a given plan."""
    
#     # Get the current task step
#     _step = _get_current_task(state)
#     _, step_name, agent, agent_input = state["steps"][_step - 1]

#     # Prepare the results dictionary
#     _results = state.get("results", {})
    
#     # Substitute placeholders in agent_input with values from _results
#     for k, v in _results.items():
#         agent_input = agent_input.replace(k, v)

#     # Execute the correct agent based on the specified tool
#     if agent == "RAGSearcher":
#         result = rag_agent({'messages': ('user', agent_input)})  # Use rag_agent for RAGSearcher tasks
#     elif agent == "Searcher":
#         result = search_agent({'messages': ('user', agent_input)})  # Use search_agent for Searcher tasks
#     elif agent == "Coder":
#         result = code_agent({'messages': ('user', agent_input)})  # Use code_agent for Coder tasks
#     else:
#         raise ValueError(f"Unknown agent type: {agent}")

#     # Store the result for this step
#     _results[step_name] = str(result)

#     # Return the updated results dictionary
#     return {"results": _results}


# def solve(state: ReWOO):
#     plan = ""
    
#     for _plan, step_name, agent, agent_input in state["steps"]:
#         # Prepare results dictionary (if any)
#         _results = state.get("results", {})

#         # Replace placeholders in the agent_input and step_name with values from results
#         for k, v in _results.items():
#             agent_input = agent_input.replace(k, v)
#             step_name = step_name.replace(k, v)

#         # Based on the agent, call the corresponding agent node and prepare the plan
#         if agent == "RAGSearcher":
#             plan += f"Plan: {_plan}\n{step_name} = {rag_node}[{'messages': agent_input}]\n"
#         elif agent == "Searcher":
#             plan += f"Plan: {_plan}\n{step_name} = {search_node}[{'messages': agent_input}\n"
#         elif agent == "Coder":
#             plan += f"Plan: {_plan}\n{step_name} = {code_node}[{'messages': agent_input}]\n"
#         else:
#             raise ValueError(f"Unknown agent type: {agent}")

#     # Prepare the prompt with the constructed plan
#     prompt = solve_prompt.format(plan=plan, task=state["task"])
    
#     # Invoke the LLM with the formatted prompt
#     result = model.invoke(prompt)

#     # Return the result from the LLM
#     return {"result": result.content}

# def _route(state):
#     _step = _get_current_task(state)
#     if _step is None:
#         # We have executed all tasks
#         return "solve"
#     else:
#         # We are still executing tasks, loop back to the "tool" node
#         return "tool"


# from langgraph.graph import END, StateGraph, START

# # Define the StateGraph with ReWOO as the context or state manager
# graph = StateGraph(ReWOO)

# # Add the nodes (steps in the process)
# graph.add_node("plan", get_plan)  # This node generates the plan for the workflow
# graph.add_node("tool", agent_execution)  # This node executes the tools defined in the plan
# graph.add_node("solve", solve)  # This node solves the problem using the generated plan

# # Add edges between the nodes
# graph.add_edge("plan", "tool")  # After the plan is generated, execute the tools
# graph.add_edge("tool", "solve")  # After tool execution, the result is passed to solve
# graph.add_edge("solve", END)  # Once solved, we reach the end of the workflow

# # Add conditional edges based on specific conditions or routing logic
# graph.add_conditional_edges("tool", _route)  # The routing logic defines what happens after the tool execution

# # Define the starting point of the workflow
# graph.add_edge(START, "plan")

# # Compile the graph to create the workflow application
# app = graph.compile()

# for s in app.stream({"task": task}):
#     print(s)
#     print("---")

## modification  of next  plan

In [ ]:

# def get_plan(m):
#     task = m
#     result = planner.invoke(task)
    
#     matches = re.findall(regex_pattern, result.content)

#     # Convert the tuple output to a dictionary list
#     steps = [
#         {
#             "description": match[0],  # Plan description
#             "step_id": match[1],      # Step identifier
#             "agent": match[2],        # Agent used
#             "input_data": match[3]    # Input data or reference
#         }
#         for match in matches
#     ]
    
#     return {"steps": steps, "plan_string": result.content}

## rewoo + hierarchy failed

In [ ]:
# # utils
# ## router
# def create_team_supervisor(llm, system_prompt, members) -> str:
#     """An LLM-based supervisor router for ReWOO-style reasoning."""
#     options = ["FINISH"] + members
    
#     # Define a function schema for routing steps with detailed 'step' information
#     function_def = {
#         "name": "route",
#         "description": "Execute the steps of a given plan sequentially by invoking the specified agent.",
#         "parameters": {
#             "title": "routeSchema",
#             "type": "object",
#             "properties": {
#                 "next": {
#                     "title": "Next Agent",
#                     "anyOf": [
#                         {"enum": options},
#                     ],
#                 },
#                 "step": {
#                     "title": "Step Description",
#                     "type": "object",  # Change from string to object to include detailed fields
#                     "properties": {
#                         "description": {
#                             "type": "string",
#                             "description": "The description of the step to execute."
#                         },
#                         "step_id": {
#                             "type" : "string",
#                             "description": "Step identifier. Eg. #E[number]"
#                         },
#                         "agent": {
#                             "type": "string",
#                             "description": "The agent to use for this step."
#                         },
#                         "input": {
#                             "type": "string",
#                             "description": "The input required for the agent in this step."
#                         }
#                     },
#                     "required": ["description", "step_id", "agent", "input"]
#                 },
#                 # "input": {
#                 #     "title": "Agent Input",
#                 #     "type": "string",
#                 #     "description": "The exact input required for the agent in this step.",
#                 # },
#             },
#             "required": ["next", "step"],
#         },
#     }
    
#     # Define a system prompt to guide the supervisor's behavior
#     prompt = ChatPromptTemplate.from_messages(
#         [
#             ("system", system_prompt),
#             MessagesPlaceholder(variable_name="messages"),
#             (
#                 "system",
#                 "Based on the conversation and plan, determine who should act next. "
#                 "Provide the step to execute, the agent to use, and the input for the agent. "
#                 "Select one of: {options}. Use FINISH when all steps are complete."
#             ),
#         ]
#     ).partial(options=str(options), team_members=", ".join(members))
    
#     # Combine with trimmer, bind functions, and output parser
#     return (
#         prompt
#         | trimmer
#         | llm.bind_functions(functions=[function_def], function_call="route")
#         | JsonOutputFunctionsParser()
#     )

# ## basic utils
# def agent_node(state, agent, name):
#     result = agent.invoke(state)
#     return {
#         "messages": [HumanMessage(content=result["messages"][-1].content, name=name)]
#     }

# def enter_chain(message: str):
#     decomposed_query = dec.invoke(message)

#     results = {
#         "messages": [HumanMessage(content=message)],
#         "task": decomposed_query
#     }
#     return results
# # state
# from typing import List, Dict
# from typing_extensions import TypedDict
# from pprint import pprint

# class ResearchTeamState(TypedDict):
#     messages: Annotated[List[BaseMessage], operator.add]
#     team_members: List[str]

#     task: str
#     plan_string: str
#     steps: List[Dict[str, str, str, str]]
#     results: dict
#     result: str

#     next: str
#     step: str
# # orch
# import re

# from langchain_core.prompts import ChatPromptTemplate

# regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"
# prompt_template = ChatPromptTemplate.from_messages([("user", rewoo_prompt)])
# planner = prompt_template | llm


# def get_plan(state: ResearchTeamState):
#     task = state["task"]
#     result = planner.invoke({"task": task})
#     # Find all matches in the sample text
#     matches = re.findall(regex_pattern, result.content)

#     # Convert the tuple output to a dictionary list
#     steps = [
#         {
#             "description": match[0],  # Plan description
#             "step_id": match[1],      # Step identifier
#             "agent": match[2],        # Agent used
#             "input_data": match[3]    # Input data or reference
#         }
#         for match in matches
#     ]
    
#     return {"steps": steps, "plan_string": result.content}

# def _get_current_task(state: ResearchTeamState):
#     if "results" not in state or state["results"] is None:
#         return 1
#     if len(state["results"]) == len(state["steps"]):
#         return None
#     else:
#         return len(state["results"]) + 1
# rag_agent = create_react_agent(llm, tools=[qdrant_retriever_tool])
# rag_node = functools.partial(agent_node, agent=rag_agent, name="RAGSearcher")

# search_agent = create_react_agent(llm, tools=[tavily_search_tool, web_scraper_tool, arxiv_search_tool])
# search_node = functools.partial(agent_node, agent=search_agent, name="Searcher")

# code_agent = create_react_agent(llm, tools=[repl_tool])
# code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

# # Define the chatbot function
# def chatbot(state: ResearchTeamState):
#     return {"messages": [llm.invoke(state["messages"])]}

# # Define the supervisor agent
# supervisor_agent = create_team_supervisor(
#     llm,
#     system_prompt=(
#         "You are a supervisor tasked with managing and orchestrating a team of workers: "
#         "RAGSearcher, Searcher, Coder, and ChatBot. Your goal is to execute the provided plan sequentially. "
#         "Each worker will perform a task as per the step description and respond with their results and status. "
#         "When all steps are complete, respond with FINISH.\n\n"
#         "At each step:\n"
#         "- Identify the next agent to act based on the plan.\n"
#         "- Specify the step to execute and the required input.\n"
#         "- Continue until all steps are completed.\n"
#         "Ensure the responses are concise, relevant, and aligned with the task."
#     ),
#     members=["RAGSearcher", "Searcher", "Coder", "ChatBot"]
# )

# # Initialize the state graph for the research process
# research_graph = StateGraph(ResearchTeamState)

# # Add nodes for each agent
# research_graph.add_node("Search", search_node)
# research_graph.add_node("WebScraper", research_node)  # Assuming this is an existing function for the web scraper agent
# research_graph.add_node("supervisor", supervisor_agent)
# research_graph.add_node("ChatBot", chatbot)  # Adding the ChatBot node

# # Define the control flow (edges)
# research_graph.add_edge("Search", "supervisor")
# research_graph.add_edge("WebScraper", "supervisor")
# research_graph.add_edge("RAGSearcher", "supervisor")
# research_graph.add_edge("Coder", "supervisor")
# research_graph.add_edge("ChatBot", "supervisor")  # Ensure ChatBot is also routed through the supervisor

# # Add conditional edges based on supervisor's decisions
# research_graph.add_conditional_edges(
#     "supervisor",
#     lambda x: x["next"],  # Determine the next step based on supervisor's output
#     {
#         "Search": "Search",
#         "WebScraper": "WebScraper",
#         "RAGSearcher": "RAGSearcher",
#         "ChatBot": "ChatBot",
#         "Coder": "Coder",
#         "FINISH": END  # The end state when all steps are finished
#     }
# )

# # Add the initial edge to start the process
# research_graph.add_edge(START, "supervisor")

# # Compile the entire research chain
# chain = research_graph.compile()

# # Function to enter the chain state and pass initial input
# def enter_chain(message: str):
#     results = {
#         "messages": [HumanMessage(content=message)],
#     }
#     return results

# # Now you can invoke the research chain with the entered message
# research_chain = enter_chain | chain

# Rewoo team

In [2]:
from typing_extensions import TypedDict, List, Optional, Union, Dict, Annotated, Literal
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage, trim_messages
from langchain_core.documents import Document
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import END, StateGraph, START, MessagesState
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from team_tools import tavily_search_tool, arxiv_search_tool, web_scraper_tool, repl_tool
from qdrant_cloud_ops import initialize_selfquery_retriever, qdrant_vector_store
from llm_chains import decomposition_chain, requires_decomposition, rephrase_chain, get_plan_chain
from dotenv import load_dotenv
from pprint import pprint
import re
import functools
import operator
from langgraph.checkpoint.memory import MemorySaver
from token_counter import tiktoken_counter



load_dotenv()
memory = MemorySaver()


llm = ChatGroq(model='llama-3.2-90b-vision-preview')


trimmer = trim_messages(
    max_tokens=5984,
    strategy="last",
    token_counter=tiktoken_counter,
    include_system=True,
    allow_partial=False,
)

qdrant_retriever = initialize_selfquery_retriever(llm, qdrant_vector_store=qdrant_vector_store)
qdrant_retriever_tool = qdrant_retriever.as_tool(
    name="retrieve_research_paper_texts",
    description="Search and return information from the vector database containing texts of several research papers, and scholarly articles",
)
# state
decomposer_chain = decomposition_chain(llm=llm)
check_query_chain = requires_decomposition(llm=llm)
rephraser_chain = rephrase_chain(llm=llm)
planner_chain = get_plan_chain(llm=llm)

check_query_chain.invoke('what is fully homomorphic encryption?')
print(rephraser_chain.invoke('what is fully homomorphic encryption?'))

print(decomposer_chain.invoke('what is fully homomorphic encryption?, where is it used, and what are the latest research'))
p= (planner_chain.invoke({'task': 'what is fully homomorphic encryption?', 'messages': []}))
print(p)
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"


matches = re.findall(regex_pattern, p)
matches
# planner_chain.invoke({'task': 'what is positional embedding', 'messages': list(ResearchTeamState['messages'])})
class ResearchTeamState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]    
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str
def decompose_or_rephrase(state: ResearchTeamState):
    question = state['messages'][-1].content
    task = None
    try:
        status = check_query_chain.invoke(question)

        if status == 'Decompose':
            task = decomposer_chain.invoke(question)
        elif status == 'Rephrase':
            task = rephraser_chain.invoke(question)
        else:
            raise ValueError(f"Unexpected status from check_query_chain: {status}")
    except Exception as e:
        raise RuntimeError(f"Error during processing: {e}")

    return {'task': task}


def get_plan(state: ResearchTeamState):
    regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"

    task = state['task']
    messages = state['messages']
    plan = planner_chain.invoke({'task': task, 'messages': messages})

    matches = re.findall(regex_pattern, plan)
    return {"steps": matches, "plan": plan}



rag_agent = create_react_agent(llm, tools=[qdrant_retriever_tool], checkpointer=memory)
search_agent = create_react_agent(llm, tools=[tavily_search_tool, arxiv_search_tool])
code_agent = create_react_agent(llm, tools=[repl_tool])


def rag_node(state: ResearchTeamState):
    trimmed_state_messages = state['messages']
    return {"messages": [HumanMessage(content=trimmed_state_messages[-1].content, name="RagSearcher")]}

def code_node(state: ResearchTeamState):
    trimmed_state_messages = state['messages']
    return {"messages": [HumanMessage(content=trimmed_state_messages[-1].content, name="Coder")]}


def search_node(state: ResearchTeamState):
    trimmed_state_messages = state['messages']
    return {"messages": [HumanMessage(content=trimmed_state_messages[-1].content, name="Searcher")]}

def _get_current_task(state: ResearchTeamState):
    # Ensure "steps" is defined and not empty
    if "steps" not in state or not state["steps"]:
        raise ValueError("The 'steps' list in the state is empty or undefined.")

    # Handle cases where "results" is not present or empty
    if "results" not in state or state["results"] is None:
        return 1

    # Ensure the results list is valid
    if not isinstance(state["results"], dict):
        raise ValueError("'results' should be a dictionary containing step outputs.")

    # If all steps are completed, return -1 as an explicit indicator
    if len(state["results"]) == len(state["steps"]):
        return -1  # Explicitly indicate completion

    # Return the index of the next task
    return len(state["results"]) + 1



def agent_exec(state: ResearchTeamState):
    """Worker node that executes the agents accordingly for a given plan."""

    _results = (state["results"] or {}) if "results" in state else {}
    _step = _get_current_task(state)
    step_desc, step_name, agent, agent_input = state["steps"][_step - 1]

    for k, v in _results.items():
        agent_input = agent_input.replace(k, v)

    result = None

    if agent == "RagSearcher":
        result = rag_agent.invoke({"messages": [HumanMessage(content=agent_input)]})['messages'][-1].content
    elif agent == "Searcher":
        result = search_agent.invoke({"messages": [HumanMessage(content=agent_input)]})['messages'][-1].content
    elif agent == "ChatBot":
        result = llm.invoke(agent_input)
    elif agent == "Coder":
        result = code_agent.invoke({"messages": [HumanMessage(content=agent_input)]})['messages'][-1].content
    else:
        raise ValueError(f"Unknown agent type: {agent}")

    if result is None:
        raise ValueError(f"Agent {agent} did not return a result for step {step_name}")

    _results[step_name] = str(result)
    return {"results": _results}

solve_prompt = """
We have created a detailed step-by-step Plan to solve the given task and obtained corresponding answers from agents for each step in the Plan. 
Use these agent-provided answers as Evidence to craft a clear, comprehensive, and cohesive response.

Plan:  
{plan}

Using the Evidence from the answers provided for each step in the Plan, solve the given task:  
Task: {task}

Provide your response below in a well-structured and coherent format:  
Response:

"""


def solve(state: ResearchTeamState):
    plan = ""
    for _plan, step_name, agent, agent_input in state["steps"]:
        _results = (state["results"] or {}) if "results" in state else {}
        for k, v in _results.items():
            agent_input = agent_input.replace(k, v)
            step_name = step_name.replace(k, v)
        plan += f"Plan: {_plan}\n{step_name} = {agent}[{agent_input}]"
    prompt = solve_prompt.format(plan=plan, task=state["task"])
    result = llm.invoke(prompt)
    return {"result": result.content, 'messages': [result]}
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        return "solve"
    else:
        return "agent_exec"
# def enter_chain(human_message: str):
#     entry_syntax = {"messages": [HumanMessage(content=human_message)],}
#     return entry_syntax


graph = StateGraph(ResearchTeamState)

graph.add_node('decompose_or_rephrase', decompose_or_rephrase)
graph.add_node('get_plan', get_plan)
graph.add_node("agent_exec", agent_exec)
graph.add_node("solve", solve)

graph.add_edge(START, 'decompose_or_rephrase')
graph.add_edge('decompose_or_rephrase', 'get_plan')
graph.add_edge('get_plan', 'agent_exec')
graph.add_conditional_edges("agent_exec", _route)
graph.add_edge("solve", END)

research_graph_compiled = graph.compile(checkpointer=memory)
research_graph = research_graph_compiled
# from IPython.display import display, Image
# display(Image(research_graph.get_graph().draw_mermaid_png()))


config = {"configurable": {"thread_id": "2"}}
for s in research_graph.stream({"messages": [HumanMessage(content='what is fully homomorphic encryption.')]}, config=config,):
    print(s)
    print("---")

# print(s["solve"]["result"])
# # for s in rag_agent.stream({'messages': [HumanMessage(content='what is the main topic or subject of san.pdf?')]}):
# #     print(s)
# #     print('----')
# r = research_graph.get_state(config=config, subgraphs=True)
# r.values['messages']
# print(r.values['messages'][-1].content)

/tmp/ipykernel_26540/1608887471.py:39: LangChainBetaWarning: This API is in beta and may change in the future.
  qdrant_retriever_tool = qdrant_retriever.as_tool(


definition of fully homomorphic encryption
what is fully homomorphic encryption?, where is fully homomorphic encryption typically applied or used?, what are the current advancements or recent research developments in fully homomorphic encryption?
Plan: Search for information on fully homomorphic encryption using the Searcher agent to gather general knowledge from online sources. #E1 = Searcher[fully homomorphic encryption definition]
Plan: Retrieve research papers and academic articles related to fully homomorphic encryption from Arxiv and the vector database using the Searcher and RagSearcher agents. #E2 = Searcher[fully homomorphic encryption research papers], #E3 = RagSearcher[fully homomorphic encryption]
Plan: Summarize the definition and key concepts of fully homomorphic encryption based on the retrieved information. #E4 = ChatBot[Summarize #E1, #E2, #E3]
{'decompose_or_rephrase': {'task': 'define fully homomorphic encryption.'}}
---
{'get_plan': {'steps': [('Search for academic 

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Empty request"}')


{'agent_exec': {'results': {'#E1': 'Fully homomorphic encryption is an encryption method that allows computations to be performed on encrypted data without decrypting it first. It is a cryptographic technique that enables computations on encrypted data while maintaining the confidentiality of the data. In the context of quantum information processing, quantum fully homomorphic encryption (QFHE) is a technique that allows computations on encrypted quantum data.', '#E2': 'It seems like the function is not working as expected. I will change the approach and provide a general definition of homomorphic encryption.\n\nHomomorphic encryption is a form of encryption that allows computations to be performed on ciphertext (encrypted data) without decrypting it first. This means that encrypted data can be processed and analyzed without compromising the security of the data, as the computations are performed directly on the encrypted data.\n\nHowever, if you are looking for a more specific definit

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

# get current user


In [ ]:
def get_current_user(request: Request):
    # Retrieve the token from cookies
    token = request.cookies.get("auth_token")
    if not token:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Not authenticated",
            headers={"WWW-Authenticate": "Bearer"},
        )

    try:
        # Decode the JWT token with automatic expiration verification
        payload = decode(
            token, SECRET_KEY, algorithms=[ALGORITHM], options={"verify_exp": True}
        )
        user_id: str = payload.get("user_id")
        email: str = payload.get("email")

        # Validate the presence of user_id and email
        if not user_id or not email:
            raise HTTPException(
                status_code=status.HTTP_401_UNAUTHORIZED,
                detail="Invalid token: missing user_id or email",
                headers={"WWW-Authenticate": "Bearer"},
            )

        # Return user details as a structured response
        return {"user_id": user_id, "email": email}

    except ExpiredSignatureError:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Token has expired",
            headers={"WWW-Authenticate": "Bearer"},
        )
    except InvalidTokenError:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Token is invalid",
            headers={"WWW-Authenticate": "Bearer"},
        )
    except Exception as e:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail=f"Error decoding token: {str(e)}",
            headers={"WWW-Authenticate": "Bearer"},
        )

# middleware code

In [ ]:
# Define the UserAuthMiddleware class
class UserAuthMiddleware(BaseHTTPMiddleware):
    def __init__(self, app, secret_key: str, algorithm: str):
        super().__init__(app)
        self.secret_key = secret_key
        self.algorithm = algorithm

    async def dispatch(self, request: Request, call_next):
        unprotected_routes = [
            "/api/login",
            "/api/signup",
            "/api/logout",
            "/api/docs",
            "/api/openapi.json",
        ]

        # Allow unprotected routes without authentication
        for route in unprotected_routes:
            if re.fullmatch(route, request.url.path):
                response = await call_next(request)
                return response

        # Example regex for routes to skip authentication
        # if re.fullmatch(r"/api/.*/\w+", request.url.path):
        if re.fullmatch(r"/api/[\w-]+/\{([\w-]+)\}(/[\w.-]+)?", request.url.path):
            response = await call_next(request)
            return response

        # Extract token from cookies
        token = request.cookies.get("auth_token")
        if not token:
            raise HTTPException(status_code=401, detail="Not authenticated")

        try:
            payload = decode(token, self.secret_key, algorithms=[self.algorithm])
            exp = payload.get("exp")
            if exp and datetime.utcfromtimestamp(exp) < datetime.utcnow():
                raise HTTPException(status_code=401, detail="Token has expired")

            # Add user data to request state
            user = {
                "user_id": payload.get("user_id"),
                "email": payload.get("email"),
            }
            request.state.user = user

        except ExpiredSignatureError as e:
            raise HTTPException(status_code=401, detail=f"Token expired: {str(e)}")
        except InvalidTokenError as e:
            raise HTTPException(status_code=401, detail=f"Invalid token: {str(e)}")
        except DecodeError as e:
            raise HTTPException(status_code=400, detail=f"Malformed token: {str(e)}")
        except Exception as e:
            raise HTTPException(status_code=500, detail=f"Unexpected error: {str(e)}")

        # Continue to the next middleware or route handler
        response = await call_next(request)
        return response

# Add UserAuthMiddleware to the application
app.add_middleware(
    UserAuthMiddleware,
    secret_key=SECRET_KEY,
    algorithm=ALGORITHM,
)


def get_authenticated_user(request: Request):
    if not request.state.user:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Not authenticated",
        )
    return request.state.user

